In [86]:
from requests_oauthlib import OAuth1Session
from requests.exceptions import ConnectionError, ReadTimeout, SSLError
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import MeCab as mc
from elasticsearch import Elasticsearch
import json, datetime, time, pytz, re, sys, traceback, unicodedata, pymongo
from pymongo import MongoClient
from collections import defaultdict
import numpy as np

In [56]:
t = mc.Tagger('-Ochasen -d /usr/lib64/mecab/dic/mecab-ipadic-neologd/')
t.parse('')
sentence = '今日は良い天気ですが、雨ですね。キャラメルマキアートがうまいです。'
node = t.parseToNode(sentence)   # 先頭行はヘッダのためスキップ

while(node):
    if node.surface != "":
        print(node.surface +"\t"+ node.feature)
    node = node.next
    if node is None:
        break

今日	名詞,副詞可能,*,*,*,*,今日,キョウ,キョー
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
良い	形容詞,自立,*,*,形容詞・アウオ段,基本形,良い,ヨイ,ヨイ
天気	名詞,一般,*,*,*,*,天気,テンキ,テンキ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
が	助詞,接続助詞,*,*,*,*,が,ガ,ガ
、	記号,読点,*,*,*,*,、,、,、
雨	名詞,一般,*,*,*,*,雨,アメ,アメ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
ね	助詞,終助詞,*,*,*,*,ね,ネ,ネ
。	記号,句点,*,*,*,*,。,。,。
キャラメルマキアート	名詞,固有名詞,一般,*,*,*,キャラメルマキアート,キャラメルマキアート,キャラメルマキアート
が	助詞,格助詞,一般,*,*,*,が,ガ,ガ
うまい	形容詞,自立,*,*,形容詞・アウオ段,基本形,うまい,ウマイ,ウマイ
です	助動詞,*,*,*,特殊・デス,基本形,です,デス,デス
。	記号,句点,*,*,*,*,。,。,。


In [59]:
KEYS = { # 自分のアカウントで入手したキーを下記に記載
        'consumer_key':'en35U31mW3P9EAqZvR6KXkZ2h',
        'consumer_secret':'JWoHyJU5KRnbeWjb17hVHAQsiPFgwGwvBkZSkRsARrApZqNEKd',
        'access_token':'868744152753647617-ei1ptGYVwJqgv2zy7f09fPVuFrsYbLY',
        'access_secret':'VHQQ1g1MDTUMRufrF2mojkxiig47qEc7ghcGj32fyMX1f',
       }
twitter = None
connect = None
db      = None
tweetdata = None
meta    = None

In [63]:
def initialize(): # twitter接続情報や、mongoDBへの接続処理等initial処理実行
    global twitter, twitter, connect, db, tweetdata, meta
    twitter = OAuth1Session(KEYS['consumer_key'],KEYS['consumer_secret'],
                            KEYS['access_token'],KEYS['access_secret'])
#   connect = Connection('localhost', 27017)     # Connection classは廃止されたのでMongoClientに変更 
    connect = MongoClient('localhost', 27017)
    db = connect.trend
    tweetdata = db.recipe_tweetdata
    meta = db.recipe_metadata
initialize()

In [81]:
def mecab_analysis(sentence):
    t = mc.Tagger('-Ochasen -d /usr/lib64/mecab/dic/mecab-ipadic-neologd/')
    t.parse('')#python3で起こるバグ対策
    sentence = sentence.replace('\n', ' ')
    
    node = t.parseToNode(sentence) 
    result_dict = defaultdict(list)
    for i in range(140):  # ツイートなのでMAX140文字
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            if word_type in ["名詞", "形容詞", "動詞"]:
                plain_word = node.feature.split(",")[6]
                if plain_word !="*":
                    result_dict[word_type].append(plain_word)
        node = node.next
        if node is None:
            break
    return result_dict

In [83]:
for d in tweetdata.find({},{'_id':1, 'id':1, 'text':1,'noun':1,'verb':1,'adjective':1,'adverb':1}):
    res = mecab_analysis(unicodedata.normalize('NFKC', d['text'])) # 半角カナを全角カナに

    # 品詞毎にフィールド分けして入れ込んでいく
    for k in res.keys():
        if k == u'形容詞': # adjective  
            adjective_list = []    
            for w in res[k]:                adjective_list.append(w)
            tweetdata.update({'_id' : d['_id']},{'$push': {'adjective':{'$each':adjective_list}}})
        elif k == u'動詞': # verb
            verb_list = []
            for w in res[k]:
                verb_list.append(w)
            tweetdata.update({'_id' : d['_id']},{'$push': {'verb':{'$each':verb_list}}})
        elif k == u'名詞': # noun
            noun_list = []
            for w in res[k]:                noun_list.append(w)
            tweetdata.update({'_id' : d['_id']},{'$push': {'noun':{'$each':noun_list}}})
        elif k == u'副詞': # adverb
            adverb_list = []
            for w in res[k]:
                adverb_list.append(w)
            tweetdata.update({'_id' : d['_id']},{'$push': {'adverb':{'$each':adverb_list}}})
    # 形態素解析済みのツイートにMecabedフラグの追加
    tweetdata.update({'_id' : d['_id']},{'$set': {'mecabed':True}})


/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  
/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.
  if __name__ == '__main__':
/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:25: DeprecationWarning: update is deprecated. Use replace_one, update_one or update_many instead.


In [89]:
connect = MongoClient('localhost', 27017)
db = connect.starbucks
tweetdata = db.tweetdata


def str_to_date_jp_utc(str_date):
    if str_date is not None:
        return datetime.datetime.strptime(str_date,'%Y-%m-%d %H:%M:%S') - datetime.timedelta(hours=9)
    else:
        return None

In [90]:
def get_mecabed_strings(from_date_str=None, to_date_str=None,include_rt=False):
    tweet_list = []
    tweet_texts = []

    from_date = str_to_date_jp_utc(from_date_str)
    to_date = str_to_date_jp_utc(to_date_str) 

    # 取得対象期間の条件設定
    if (from_date_str is not None) and (to_date_str is not None):
        query = {'created_datetime':{"$gte":from_date, "$lt":to_date}}
    elif (from_date_str is not None) and (to_date_str is None):
        query = {'created_datetime':{"$gte":from_date}}
    elif (from_date_str is None) and (to_date_str is not None):
        query = {'created_datetime':{"$lt":to_date}}
    else:
        query = {}

    # spam除去
    query['spam'] = None

    # リツイートを含むか否か
    if include_rt == False:
        query['retweeted_status'] = None
    else:
        query['retweeted_status'] = {"$ne": None}

    # 指定した条件のツイートを取得
    for d in tweetdata.find(query,{'noun':1, 'verb':1, 'adjective':1, 'adverb':1,'text':1}):
        tweet = ""
        # Mecabで分割済みの単語をのリストを作成
        if 'noun' in d:
            for word in d['noun']:
                tweet += word + " "
        if 'verb' in d:
            for word in d['verb']:
                tweet += word + " "
        if 'adjective' in d:
            for word in d['adjective']:
                tweet += word + " "
        if 'adverb' in d:
            for word in d['adverb']:
                tweet += word + " "
        tweet_list.append(tweet)
        tweet_texts.append(d['text'])
    return {"tweet_list":tweet_list,"tweet_texts":tweet_texts}

In [91]:
# 全期間
ret_all = get_mecabed_strings()
tw_list_all = ret_all['tweet_list']
c_vec = CountVectorizer(stop_words=[u"料理"])  # 「スタバ」は全Tweetに含まれるので除外
c_vec.fit(tw_list_all)                   # 全Tweetに含まれる単語の集合をここでセット
c_terms = c_vec.get_feature_names()             # 各ベクトル要素に対応する単語を表すベクトル

# 期間の前後でひとまとまりと考え、transformする
transformed = c_vec.transform([' '.join(tw_list_before),' '.join(tw_list_after)])

ValueError: empty vocabulary; perhaps the documents only contain stop words

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
